---
# **Thesis Research Project Code**
### Independent Virtual Reality Training on Public Speaking Skills of University Students: An Experimental Study
---

Michael Matheo Kallas - i6283098

📧 m.kallas@student.maastrichtuniversity.nl

---

In [2]:
import speech_recognition as sr
import pandas as pd
import numpy as np
import syllapy
import librosa
import os

---

# GSR

#### Computing Means of Moving Medians

In [ ]:
def compute_moving_median(file_path, window_size):
    data = pd.read_csv(file_path)
    data['Time'] = pd.to_datetime(data['Time (ms)'], unit='ms')
    data.set_index('Time', inplace=True)
    data['Moving Median'] = data['Skin_Conductance (uS)'].rolling(window=window_size).median()
    mean_of_moving_median = data['Moving Median'].mean()
    data['Time Numeric'] = (data.index - data.index[0]).total_seconds() 
    return mean_of_moving_median

results_df = pd.DataFrame(columns=['File', 'Mean of Moving Median'])
window_size = '3000ms'

for id_number in range(1, 25):  
    for trial in ['G1', 'G2']:  
        file_name = f'{trial}{id_number:02}.csv'
        mean_of_moving_median = compute_moving_median(file_name, window_size)
        new_row = pd.DataFrame({
            'File': [file_name], 
            'Mean of Moving Median': [mean_of_moving_median],
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

results_df.to_excel('GSR_Mean.xlsx', index=False)
display(results_df)

# Vocal Attributes

## f0 Parameters

#### Marking IPU Boundaries

In [ ]:
gaps_data = []

for i in range(1, 25):
    for identifier in ['M1', 'M2']:
        file_number = f'{i:02}'
        audio_file = f'{file_number}{identifier}.wav'
        signal, sampling_rate = librosa.load(audio_file)
        intensity_envelope = librosa.feature.rms(y=signal)[0]
        intensity_db = librosa.amplitude_to_db(intensity_envelope, ref=np.max)
        times = librosa.frames_to_time(np.arange(len(intensity_db)), sr=sampling_rate)
        silent_gaps = intensity_db < -25
        gap_indices = np.where(silent_gaps)[0]
        gap_regions = np.split(gap_indices, np.where(np.diff(gap_indices) != 1)[0] + 1)
        for gap in gap_regions:
            if gap.size > 0:
                start_time = times[gap[0]]
                end_time = times[gap[-1]]
                gap_duration = end_time - start_time
                if gap_duration >= 0.2:  
                    gaps_data.append({'File': audio_file, 'Start Time': start_time, 'End Time': end_time})

silent_gaps_df = pd.DataFrame(gaps_data)
silent_gaps_df.to_csv('silent_gaps.csv', index=False)
display(silent_gaps_df)

#### Filtering f0 Output

In [ ]:
def filter_f0_data(file_path, silent_gaps):
    f0_data = pd.read_csv(file_path, delim_whitespace=True)
    keep_mask = pd.Series(True, index=f0_data.index)
    for _, row in silent_gaps.iterrows():
        start_time = row['Start Time']
        end_time = row['End Time']
        keep_mask = keep_mask & ((f0_data['Time'] < start_time) | (f0_data['Time'] > end_time))
    filtered_data = f0_data[keep_mask]
    filtered_data.to_csv(file_path.replace('.f0', '_filtered.txt'), sep='\t')

for i in range(1, 25):
    for identifier in ['M1', 'M2']:
        file_number = f'{i:02}'
        audio_file = f'{file_number}{identifier}'
        file_path = f'{audio_file}.f0'
        file_specific_gaps = silent_gaps_df[silent_gaps_df['File'].str.contains(audio_file)]        
        filter_f0_data(file_path, file_specific_gaps)

#### Calculating f0 Statistics

In [11]:
def hz_to_semitones(f0_values):
    semitones = 12 * np.log2(f0_values / 100)
    return semitones

def calculate_f0_stats(file_path):
    f0_data = pd.read_csv(file_path, delim_whitespace=True)
    f0_data['Semitones'] = hz_to_semitones(f0_data['F0'])
    stats = {
        'Mean Semitones': f0_data['Semitones'].mean(),
        'STD Semitones': f0_data['Semitones'].std()
    }
    return stats

filtered_data = [f for f in os.listdir() if '_filtered.txt' in f]
results = []

for file in filtered_data:
    file_stats = calculate_f0_stats(file)
    if file_stats:
        file_stats['File'] = file
        results.append(file_stats)

results_df = pd.DataFrame(results)
results_df.to_excel('f0_statistics.xlsx', index=False)
display(results_df)

## Tempo Parameters

#### Detecting Silence Durations

In [ ]:
def detect_silences(audio_file):
    signal, sampling_rate = librosa.load(audio_file)
    intensity_envelope = librosa.feature.rms(y=signal)[0]
    intensity_db = librosa.amplitude_to_db(intensity_envelope, ref=np.max)
    times = librosa.frames_to_time(np.arange(len(intensity_db)), sr=sampling_rate)
    silent_gaps = intensity_db < - 25
    gap_indices = np.where(silent_gaps)[0]
    if len(gap_indices) == 0:
        return 0.0  
    gap_regions = np.split(gap_indices, np.where(np.diff(gap_indices) != 1)[0] + 1)
    total_silence_duration = 0.0
    for gap in gap_regions:
        if len(gap) > 0:
            start_time = times[gap[0]]
            end_time = times[gap[-1]]
            gap_duration = end_time - start_time
            if gap_duration >= 0.3:
                total_silence_duration += gap_duration
    return total_silence_duration

audio_directory = os.getcwd()
audio_files = []
for i in range(1, 25):
    for m in ['M1', 'M2']:
        audio_files.append(f'{i:02d}{m}.wav')
results = []

for audio_file in audio_files:
    file_path = os.path.join(audio_directory, audio_file)
    if os.path.isfile(file_path):
        silence_duration = detect_silences(file_path)
        results.append({
            'Filename': audio_file,
            'Total Silence Duration': silence_duration
        })

silences_df = pd.DataFrame(results)
display(silences_df)

#### Transcribing Audio Files

In [10]:
def transcribe_audio(file_path):
    with sr.AudioFile(file_path) as source:
        audio = sr.Recognizer().record(source) 
        return sr.Recognizer().recognize_google(audio)

directory = os.getcwd()

transcript_file = 'transcripts.txt'
with open(transcript_file, 'w', encoding='utf-8') as file_out:
    for filename in os.listdir(directory):
        if filename.endswith('.wav'):
            file_path = os.path.join(directory, filename)
            transcript = transcribe_audio(file_path)
            file_out.write(f'Filename: {filename}\nTranscript: {transcript}\n\n')

#### Counting Number of Syllables

In [ ]:
def extract_transcripts(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read().split('Filename: ')
    transcripts = {}
    for entry in content[1:]:  
        lines = entry.strip().split('\n')
        filename = lines[0].strip()  
        transcript = ' '.join(lines[1:])  
        transcripts[filename] = transcript.split('Transcript: ')[1].strip()
    return transcripts

def count_total_syllables(transcripts):
    syllable_counts = []
    for filename, transcript in transcripts.items():
        syllable_count = 0
        words = transcript.split()
        for word in words:
            syllable_count += syllapy.count(word.lower())
        syllable_counts.append({'Filename': filename, 'Total Syllables': syllable_count})
    return syllable_counts

file_path = 'transcripts.txt'
transcripts = extract_transcripts(file_path)
syllable_data = count_total_syllables(transcripts)
syllable_df = pd.DataFrame(syllable_data)
display(syllable_df)

#### Calculating Tempo Statistics

In [ ]:
def calculate_tempo_stats(file, silences_df, syllable_df):
    audio_file = os.path.join(audio_directory, file)
    signal, sampling_rate = librosa.load(audio_file)
    total_time_of_presentation = min(librosa.get_duration(y=signal, sr=sampling_rate), 120)
    silence_info = silences_df[silences_df['Filename'] == file]
    syllable_info = syllable_df[syllable_df['Filename'] == file]
    total_silence_duration = silence_info['Total Silence Duration'].values[0]
    total_syllables = syllable_info['Total Syllables'].values[0]
    total_speaking_time = total_time_of_presentation - total_silence_duration
    speech_rate = total_syllables / total_time_of_presentation if total_time_of_presentation > 0 else 0
    asd = total_speaking_time / total_syllables if total_syllables > 0 else 0
    return {
        'Filename': file,
        'Presentation Time': total_time_of_presentation,
        'Speech Rate': speech_rate,
        'ASD': asd
    }

audio_directory = os.getcwd()
files = []
for i in range(1, 25):
    for m in ['M1', 'M2']:
        files.append(f'{i:02d}{m}.wav')
results = [calculate_tempo_stats(filename, silences_df, syllable_df) for filename in files]

results_df = pd.DataFrame(results)
results_df.to_excel('tempo_statistics.xlsx', index=False)
display(results_df)

---